In [1]:
from transformers import AutoModelForMaskedLM, AutoConfig
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import pandas as pd
import torch
from torch import nn
from time import time
from torch.autograd import Variable
import torch.nn.functional as F
from torch.optim import Adadelta

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/dberta-base-model/rust_model.ot
/kaggle/input/dberta-base-model/config.json
/kaggle/input/dberta-base-model/merges.txt
/kaggle/input/dberta-base-model/vocab.json
/kaggle/input/dberta-base-model/tf_model.h5
/kaggle/input/dberta-base-model/tokenizer_config.json
/kaggle/input/dberta-base-model/bpe_encoder.bin
/kaggle/input/dberta-base-model/pytorch_model.bin
/kaggle/input/cnn-sent-mod/CNN_SENT_MODEL.bin
/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv
/kaggle/input/big-embedder/1_epoch_mlm.bin


In [2]:
class MLMCONFIG:
    SOURCE = 'https://www.kaggle.com/code/chaitanyagiri/deberta-pre-training-using-mlm'
    TRAIN = "/kaggle/input/feedback-prize-english-language-learning/train.csv"
    TEST = "/kaggle/input/feedback-prize-english-language-learning/test.csv"
    MASKING = 0.10
    MAX_LEN = 356
    CLS_TOKEN = 1
    PAD_TOKEN = 0
    SEP_TOKEN = 2
    MASK_TOKEN = 50264
    BATCH_SIZE = 8
    TEST_BATCH_SIZE = 1
    EPOCHS = 1
    LR = 1e-5
    MODEL_PATH = "/kaggle/input/dberta-base-model/"
    SAVE_PATH = "/kaggle/working/fine-tuned-mlm.bin"
    FINE_TUNED = "/kaggle/input/big-embedder/1_epoch_mlm.bin"
    NUM_WORKERS = 4

    
class CNN_CONFIG:
    TRAIN = "/kaggle/input/feedback-prize-english-language-learning/train.csv"
    TEST = "/kaggle/input/feedback-prize-english-language-learning/test.csv"
    LR = 0.001
    TRAIN_BATCH_SIZE = 32
    TEST_BATCH_SIZE = 8
    EPOCHS = 20
    N_FOLDS = 4
    TARGETS = ['cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions']
    PADDING = False
    NUM_WORKERS = 4
    FINE_TUNED = "/kaggle/input/cnn-sent-mod/CNN_SENT_MODEL.bin"
  

In [3]:
class TrainMLMDatasetup(Dataset):
    def __init__(self, data):
        self.data = data["full_text"]
        self.tokenizer = AutoTokenizer.from_pretrained(MLMCONFIG.MODEL_PATH)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        text = self.data[item]
        inputs = self.tokenizer.encode_plus(
            text,
            return_tensors='pt',
            add_special_tokens=True,
            max_length=MLMCONFIG.MAX_LEN,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
        )

        # copy input ids to create out labels tensor
        inputs["labels"] = inputs.input_ids.detach().clone()

        # create a set of floats and mask anything below our masking %
        # include logic to avoid masking the [CLS] [PAD], & [SEP] tokens
        rand = torch.rand(inputs.input_ids.shape)
        masked_arr = (
            (rand < MLMCONFIG.MASKING)
            * (inputs.labels != MLMCONFIG.CLS_TOKEN)
            * (inputs.labels != MLMCONFIG.SEP_TOKEN)
            * (inputs.labels != MLMCONFIG.PAD_TOKEN)
        )

        # get the non-zeros from the masked array
        selection = []
        for idx in range(masked_arr.shape[0]):
            selection.append(torch.flatten(masked_arr[idx].nonzero()).tolist())

        # use selection to mask the input_ids based upon the config.MASKING set
        for idx in range(masked_arr.shape[0]):
            inputs.input_ids[idx, selection[idx]] = MLMCONFIG.MASK_TOKEN

        input_ids = torch.flatten(inputs.input_ids)
        attention_mask = torch.flatten(inputs.attention_mask)
        labels = torch.flatten(inputs.labels)
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

    
    
    
class TestMLMDatasetup(Dataset):
    def __init__(self, data):
        self.data = data["full_text"]
        self.tokenizer = AutoTokenizer.from_pretrained(MLMCONFIG.MODEL_PATH)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        text = self.data[item]
        inputs = self.tokenizer.encode_plus(
            text,
            return_tensors='pt',
            add_special_tokens=True,
            max_length=MLMCONFIG.MAX_LEN,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
        )

        input_ids = torch.flatten(inputs.input_ids)
        attention_mask = torch.flatten(inputs.attention_mask)
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
        }


In [4]:
class MLMFineTuner(nn.Module):
    def __init__(self):
        super(MLMFineTuner, self).__init__()
        self.epochs = MLMCONFIG.EPOCHS
        self.learning_rate = MLMCONFIG.LR
        self.config = AutoConfig.from_pretrained(MLMCONFIG.MODEL_PATH, output_hidden_states=True)
        self.model = AutoModelForMaskedLM.from_pretrained(MLMCONFIG.MODEL_PATH, config=self.config)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")

    def fit(self, data):

        train_dataset = TrainMLMDatasetup(data)
        train_dataloader = DataLoader(train_dataset, batch_size=MLMCONFIG.BATCH_SIZE, shuffle=True, num_workers=MLMCONFIG.NUM_WORKERS)
        self.model.to(self.device)
        self.model.train()
        optim = AdamW(self.model.parameters(), lr=MLMCONFIG.LR)
        for epoch in range(self.epochs):
            for idx, batch in enumerate(train_dataloader):
                self.model.zero_grad()
                input_ids = batch['input_ids'].to(self.device)
                labels = batch['labels'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                output = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = output.loss
                print(f"epoch {epoch} --- step {idx} --- step size {input_ids.shape[0]} loss {loss.item()}")
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                loss.backward()
                optim.step()
            torch.save(self.model.state_dict(), MLMCONFIG.SAVE_PATH)
        
    def get_embeddings(self, X):
        test_dataset = TestMLMDatasetup(data=X)
        test_loader = DataLoader(
            test_dataset,
            batch_size=MLMCONFIG.TEST_BATCH_SIZE,
            shuffle=False,
            num_workers=MLMCONFIG.NUM_WORKERS,
            pin_memory=True,
            drop_last=False,
        )

        self.model.load_state_dict(
            torch.load(MLMCONFIG.FINE_TUNED)
        )
        self.model.to(self.device)
        self.model.eval()
        token_vectors = []
        for idx, batch in enumerate(tqdm(test_loader)):
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            outputs = self.model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            hidden_states = outputs.hidden_states
            token_vectors.append(hidden_states[-2][0].cpu().detach())
        return token_vectors
        

In [5]:
def targets_to_tensor(df, target_columns):
    return torch.tensor(df[target_columns].values, dtype=torch.float32)

In [6]:
TRAIN = pd.read_csv(MLMCONFIG.TRAIN) 
TEST = pd.read_csv(MLMCONFIG.TEST) 

In [7]:
run_mlm = 0
if run_mlm:
    mlm = MLMFineTuner()
    mlm.fit(data=TRAIN)


In [8]:
class TrainEmbeddingDataset(Dataset):
    def __init__(self, data):
        mlm = MLMFineTuner()
        self.data = mlm.get_embeddings(X=data)
        self.targets = targets_to_tensor(df=data,target_columns=CNN_CONFIG.TARGETS)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        record = self.data[item]
        target = self.targets[item]
        
        return {
            "input_ids": record,
            "labels": torch.tensor(target, dtype=torch.long),
        }

    
class TestEmbeddingDataset(Dataset):
    def __init__(self, data):
        mlm = MLMFineTuner()
        self.data = mlm.get_embeddings(X=data)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        record = self.data[item]
        
        return {
            "input_ids": record,
        }

In [9]:
class SentCNN(nn.Module):
    def __init__(self):
        super(SentCNN, self).__init__()
        dropout = 0.5
        static = True
        V = MLMCONFIG.MAX_LEN
        D = 768
        C = 6
        Co = 3
        Ks = [3, 4, 5]
        
        self.static = static
        self.embed = nn.Embedding(V, D)
        self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, (K, D)) for K in Ks])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(Ks) * Co, 6)

    def forward(self, x):
        x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc(x)  # (N, C)
        return logit

In [10]:
class SentenceCNNTrainer(nn.Module):
    def __init__(self):
        super(SentenceCNNTrainer, self).__init__()
        self.model = SentCNN()
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
    
    def fit(self, data):
        trainembedderset = TrainEmbeddingDataset(data=data)
        train_dataloader = DataLoader(
            trainembedderset,
            batch_size=CNN_CONFIG.TRAIN_BATCH_SIZE,
            shuffle=True,
            num_workers=CNN_CONFIG.NUM_WORKERS,
            pin_memory=True,
            drop_last=True,
        )
        criterion = nn.SmoothL1Loss(reduction='mean')
        self.model.to(self.device)
        self.model.train()
        optimizer = AdamW(params=self.model.parameters(), lr=CNN_CONFIG.LR)
        for epoch in range(CNN_CONFIG.EPOCHS):
            for step, batch in enumerate(train_dataloader):
                self.model.zero_grad()
                input_ids = batch["input_ids"].to(self.device)
                labels = batch["labels"].to(self.device)
                prediction_probas = self.model(input_ids)
                loss = criterion(prediction_probas, labels)
                loss.backward()
                print(f"epoch {epoch} --- step {step} --- step size {input_ids.shape[0]} loss {loss.item()}")
                # Update model parameters:
                # fine tune BERT params and train additional dense layers
                optimizer.step()
                # update learning rate
            torch.save(self.model.state_dict(), "CNN_SENT_MODEL.bin")
    def predict(self, data):
        testembedderset = TestEmbeddingDataset(data=data)
        test_dataloader = DataLoader(
            testembedderset,
            batch_size=1,
            shuffle=True,
            num_workers=CNN_CONFIG.NUM_WORKERS,
            pin_memory=True,
            drop_last=True,
        )
        self.model.eval()
        predictions = []
        for step, batch in enumerate(tqdm(test_dataloader)):
            input_ids = batch["input_ids"].to(self.device)
            with torch.no_grad():
                pred_probas = self.model(input_ids)
                
            predictions.append(pred_probas.cpu().detach().numpy())
        predictions = pd.DataFrame(np.concatenate(predictions))
        predictions.columns = CNN_CONFIG.TARGETS
        return predictions



In [11]:
clf = SentenceCNNTrainer()
clf.fit(data=TRAIN)

Some weights of the model checkpoint at /kaggle/input/dberta-base-model/ were not used when initializing DebertaForMaskedLM: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at /kaggle/input/dberta-base-model/ and are newly initialized: ['cls.predictions.

epoch 0 --- step 0 --- step size 32 loss 2.7028234004974365
epoch 0 --- step 1 --- step size 32 loss 2.340407371520996
epoch 0 --- step 2 --- step size 32 loss 2.3022775650024414
epoch 0 --- step 3 --- step size 32 loss 2.3712892532348633
epoch 0 --- step 4 --- step size 32 loss 2.304643154144287
epoch 0 --- step 5 --- step size 32 loss 2.1172547340393066
epoch 0 --- step 6 --- step size 32 loss 2.2287089824676514
epoch 0 --- step 7 --- step size 32 loss 2.2328388690948486
epoch 0 --- step 8 --- step size 32 loss 2.1692614555358887
epoch 0 --- step 9 --- step size 32 loss 2.2546162605285645
epoch 0 --- step 10 --- step size 32 loss 2.159183979034424
epoch 0 --- step 11 --- step size 32 loss 2.151548385620117
epoch 0 --- step 12 --- step size 32 loss 2.0133614540100098
epoch 0 --- step 13 --- step size 32 loss 1.900999903678894
epoch 0 --- step 14 --- step size 32 loss 2.2243897914886475
epoch 0 --- step 15 --- step size 32 loss 2.0869388580322266
epoch 0 --- step 16 --- step size 32 lo

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construc

epoch 1 --- step 0 --- step size 32 loss 1.4548826217651367
epoch 1 --- step 1 --- step size 32 loss 1.4775099754333496
epoch 1 --- step 2 --- step size 32 loss 1.5124086141586304
epoch 1 --- step 3 --- step size 32 loss 1.6611522436141968
epoch 1 --- step 4 --- step size 32 loss 1.5162461996078491
epoch 1 --- step 5 --- step size 32 loss 1.444655418395996
epoch 1 --- step 6 --- step size 32 loss 1.4352641105651855
epoch 1 --- step 7 --- step size 32 loss 1.488265037536621
epoch 1 --- step 8 --- step size 32 loss 1.5790613889694214
epoch 1 --- step 9 --- step size 32 loss 1.3008599281311035
epoch 1 --- step 10 --- step size 32 loss 1.5947318077087402
epoch 1 --- step 11 --- step size 32 loss 1.4795112609863281
epoch 1 --- step 12 --- step size 32 loss 1.3173869848251343
epoch 1 --- step 13 --- step size 32 loss 1.3626420497894287
epoch 1 --- step 14 --- step size 32 loss 1.5544264316558838
epoch 1 --- step 15 --- step size 32 loss 1.3862273693084717
epoch 1 --- step 16 --- step size 32

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construc

...

epoch 17 --- step 0 --- step size 32 loss 0.40716981887817383
epoch 17 --- step 1 --- step size 32 loss 0.3894590139389038
epoch 17 --- step 2 --- step size 32 loss 0.34579038619995117
epoch 17 --- step 3 --- step size 32 loss 0.5177112221717834
epoch 17 --- step 4 --- step size 32 loss 0.41602903604507446
epoch 17 --- step 5 --- step size 32 loss 0.3798186182975769
epoch 17 --- step 6 --- step size 32 loss 0.4725450873374939
epoch 17 --- step 7 --- step size 32 loss 0.281680166721344
epoch 17 --- step 8 --- step size 32 loss 0.4046476483345032
epoch 17 --- step 9 --- step size 32 loss 0.44176894426345825
epoch 17 --- step 10 --- step size 32 loss 0.45880016684532166
epoch 17 --- step 11 --- step size 32 loss 0.4335342347621918
epoch 17 --- step 12 --- step size 32 loss 0.3637373447418213
epoch 17 --- step 13 --- step size 32 loss 0.43227753043174744
epoch 17 --- step 14 --- step size 32 loss 0.42169734835624695
epoch 17 --- step 15 --- step size 32 loss 0.43780678510665894
epoch 17 --

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construc

epoch 18 --- step 0 --- step size 32 loss 0.3607901930809021
epoch 18 --- step 1 --- step size 32 loss 0.31305575370788574
epoch 18 --- step 2 --- step size 32 loss 0.3599469065666199
epoch 18 --- step 3 --- step size 32 loss 0.3265160620212555
epoch 18 --- step 4 --- step size 32 loss 0.414347380399704
epoch 18 --- step 5 --- step size 32 loss 0.3421704173088074
epoch 18 --- step 6 --- step size 32 loss 0.3872523009777069
epoch 18 --- step 7 --- step size 32 loss 0.4359002709388733
epoch 18 --- step 8 --- step size 32 loss 0.23913463950157166
epoch 18 --- step 9 --- step size 32 loss 0.4013979434967041
epoch 18 --- step 10 --- step size 32 loss 0.3895721435546875
epoch 18 --- step 11 --- step size 32 loss 0.444241464138031
epoch 18 --- step 12 --- step size 32 loss 0.3094918429851532
epoch 18 --- step 13 --- step size 32 loss 0.3495004177093506
epoch 18 --- step 14 --- step size 32 loss 0.42758092284202576
epoch 18 --- step 15 --- step size 32 loss 0.3877057433128357
epoch 18 --- step

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construc

epoch 19 --- step 0 --- step size 32 loss 0.35611170530319214
epoch 19 --- step 1 --- step size 32 loss 0.35133689641952515
epoch 19 --- step 2 --- step size 32 loss 0.3484143018722534
epoch 19 --- step 3 --- step size 32 loss 0.26366177201271057
epoch 19 --- step 4 --- step size 32 loss 0.37439775466918945
epoch 19 --- step 5 --- step size 32 loss 0.3765714764595032
epoch 19 --- step 6 --- step size 32 loss 0.3203573226928711
epoch 19 --- step 7 --- step size 32 loss 0.26533132791519165
epoch 19 --- step 8 --- step size 32 loss 0.3712722659111023
epoch 19 --- step 9 --- step size 32 loss 0.4896305203437805
epoch 19 --- step 10 --- step size 32 loss 0.3993581235408783
epoch 19 --- step 11 --- step size 32 loss 0.5821589231491089
epoch 19 --- step 12 --- step size 32 loss 0.3498574495315552
epoch 19 --- step 13 --- step size 32 loss 0.3530066907405853
epoch 19 --- step 14 --- step size 32 loss 0.393622487783432
epoch 19 --- step 15 --- step size 32 loss 0.3191549777984619
epoch 19 --- s

In [12]:

predictions = clf.predict(data=TEST)
submission = pd.merge(TEST, predictions, left_index=True,right_index=True)
submission = submission.drop('full_text',axis=1)
submission.to_csv('submission.csv', index=False)


Some weights of the model checkpoint at /kaggle/input/dberta-base-model/ were not used when initializing DebertaForMaskedLM: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at /kaggle/input/dberta-base-model/ and are newly initialized: ['cls.predictions.

In [13]:
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.745597,2.571096,2.752243,2.604530,2.616288,2.687251
1,000BAD50D026,2.479983,2.218592,2.382292,2.226262,2.300099,2.428221
2,00367BB2546B,2.659872,2.457405,2.632870,2.482533,2.514230,2.603660
